In [ ]:
import backtrader as bt
import pandas as pd
import numpy as np
import os
from glob import glob

# ----------------------
# 1. 数据加载（修复时间对齐逻辑）
# ----------------------
def load_single_minute_data(stock_code, data_dir):
    """加载单只股票的分钟数据"""
    file_path = glob(os.path.join(data_dir, f"stock_code={stock_code}", "data.parquet"))
    if not file_path:
        print(f"警告：未找到{stock_code}的分钟数据，跳过该股票")
        return None
    file_path = file_path[0]
    
    df = pd.read_parquet(file_path)
    df['datetime'] = pd.to_datetime(df['time'])
    df.set_index('datetime', inplace=True)
    df['openinterest'] = 0
    return df[['open', 'high', 'low', 'close', 'volume', 'openinterest']]

def load_correlated_group_data(correlated_stocks, data_dir, target_stock):
    """
    批量加载关联组股票数据，并与目标股票（600570）时间对齐
    修复：用循环逐步求交集，避免参数过多报错
    """
    # 先加载目标股票数据（作为时间基准）
    target_df = load_single_minute_data(target_stock, data_dir)
    if target_df is None:
        raise FileNotFoundError(f"目标股票{target_stock}数据缺失，无法继续")
    
    # 批量加载关联组其他股票数据
    group_data = {}
    for stock in correlated_stocks:
        if stock == target_stock:
            continue  # 跳过目标股票
        df = load_single_minute_data(stock, data_dir)
        if df is not None:
            group_data[stock] = df
    
    if not group_data:  # 无有效关联股票
        print("警告：无有效关联股票数据，仅使用目标股票数据")
        return target_df, {}
    
    # 修复：循环逐步求所有关联股票的时间交集（避免参数过多）
    # 第一步：取第一只关联股票的索引作为初始交集
    common_index = next(iter(group_data.values())).index
    # 第二步：循环与其他关联股票的索引求交集
    for df in group_data.values():
        common_index = common_index.intersection(df.index)
    # 第三步：再与目标股票的索引求交集（确保目标股票数据包含）
    common_index = target_df.index.intersection(common_index)
    
    # 按共同时间索引过滤数据
    target_df = target_df.loc[common_index]
    for stock, df in group_data.items():
        group_data[stock] = df.loc[common_index]
    
    print(f"关联组数据加载完成：共{len(group_data)+1}只股票（含目标股票），时间范围{common_index.min()}至{common_index.max()}")
    return target_df, group_data

# ----------------------
# 2. 自定义佣金计算器（含印花税）
# ----------------------
class StampDutyCommissionScheme(bt.CommInfoBase):
    params = (
        ('commission', 0.0002),  # 佣金0.02%
        ('stamp_duty', 0.001),   # 印花税0.1%（卖出时）
        ('stocklike', True),
        ('commtype', bt.CommInfoBase.COMM_PERC),
    )

    def _getcommission(self, size, price, pseudoexec):
        commission = abs(size) * price * self.p.commission
        if size < 0:  # 卖出加收印花税
            commission += abs(size) * price * self.p.stamp_duty
        return commission

# ----------------------
# 3. 关联组联动做T策略（核心）
# ----------------------
class CorrelatedGroupTStrategy(bt.Strategy):
    params = (
        ('target_stock', '600570.XSHG'),
        # 量价信号参数
        ('support_window', 45),  # 支撑/压力位窗口（分钟）
        ('vol_multiple', 1.2),   # 量能放大阈值
        ('dev_threshold', 0.006),# 价格偏离30分钟均线阈值（0.6%）
        # 关联组情绪参数
        ('情绪阈值', 0.6),        # 上涨个股占比≥60%才操作
        # 风险控制参数
        ('hold_max_minutes', 8), # 最大持仓时间（8分钟）
        ('t_position_ratio', 0.08),# 做T仓位（8%总资金）
        ('stop_loss', 0.008),    # 止损（0.8%）
        ('take_profit', 0.01),   # 止盈（1%）
        ('daily_max_trades', 2), # 每日最大做T次数
    )

    def __init__(self):
        # 数据引用：datas[0] = 目标股票（600570），datas[1:] = 关联组其他股票
        self.target_data = self.datas[0]
        self.correlated_datas = self.datas[1:]  # 关联组其他股票数据列表
        
        # 目标股票核心指标
        self.target_close = self.target_data.close
        self.target_vol = self.target_data.volume
        self.target_avg30 = bt.indicators.SMA(self.target_close, period=30)  # 30分钟均线
        self.target_support = bt.indicators.Lowest(self.target_close, period=self.p.support_window)  # 支撑位
        self.target_avgvol = bt.indicators.SMA(self.target_vol, period=self.p.support_window)  # 平均量能

        # 交易状态变量
        self.t_pos = 0  # 做T仓位
        self.buy_price = 0  # 买入价
        self.hold_minutes = 0  # 持仓分钟数
        self.daily_trades = 0  # 当日做T次数
        self.last_date = None  # 跨日重置标记

    def calculate_group_sentiment(self):
        """计算关联组情绪：上涨个股数量占比"""
        up_count = 0
        total_valid = 0
        for data in self.correlated_datas:
            # 跳过无成交量的股票（避免无效数据）
            if data.volume[0] == 0:
                continue
            # 上涨判断：当前分钟收盘价 > 上一分钟收盘价（确保有前一分钟数据）
            if len(data.close) > 1 and data.close[0] > data.close[-1]:
                up_count += 1
            total_valid += 1
        # 避免除以0（无有效关联股票时，默认情绪不满足）
        return up_count / total_valid if total_valid > 0 else 0

    def next(self):
        current_date = self.target_data.datetime.date(0)
        current_time = self.target_data.datetime.time(0)

        # 1. 跨日重置
        if current_date != self.last_date:
            self.daily_trades = 0
            self.last_date = current_date
            self.hold_minutes = 0

        # 2. 时间过滤：仅交易时间（9:35-11:25，13:05-14:50）
        trade_time = (
            (9 == current_time.hour and current_time.minute >= 35) or
            (10 <= current_time.hour < 11 and current_time.minute <= 25) or
            (13 == current_time.hour and current_time.minute >= 5) or
            (14 <= current_time.hour < 15 and current_time.minute <= 50)
        )
        if not trade_time:
            # 收盘前10分钟清仓
            if current_time.hour == 14 and current_time.minute > 50 and self.t_pos > 0:
                self.sell(data=self.target_data, size=self.t_pos)
                self.t_pos = 0
            return

        # 3. 计算关联组情绪（核心过滤条件）
        group_sentiment = self.calculate_group_sentiment()
        if group_sentiment < self.p.情绪阈值:  # 情绪不满足，不操作
            if self.t_pos > 0:  # 清仓现有仓位
                self.sell(data=self.target_data, size=self.t_pos)
                self.t_pos = 0
            return

        # 4. 目标股票指标计算
        target_deviation = (self.target_close[0] - self.target_avg30[0]) / self.target_avg30[0]  # 价格偏离均线
        target_vol_multiple = self.target_vol[0] / self.target_avgvol[0] if self.target_avgvol[0] != 0 else 0  # 量能倍数
        near_support = abs(self.target_close[0] - self.target_support[0]) / self.target_support[0] <= 0.003  # 接近支撑位

        # 5. 持仓监控（止盈/止损/超时）
        if self.t_pos > 0:
            self.hold_minutes += 1
            profit_rate = (self.target_close[0] - self.buy_price) / self.buy_price

            # 止盈
            if profit_rate >= self.p.take_profit or (target_deviation >= 0 and target_vol_multiple < 1):
                self.sell(data=self.target_data, size=self.t_pos)
                self.t_pos = 0
                self.hold_minutes = 0
                return

            # 止损或超时
            if profit_rate <= -self.p.stop_loss or self.hold_minutes >= self.p.hold_max_minutes:
                self.sell(data=self.target_data, size=self.t_pos)
                self.t_pos = 0
                self.hold_minutes = 0
                return
            return

        # 6. 买入信号（情绪+量价共振）
        if (near_support and  # 接近支撑位
            target_deviation <= -self.p.dev_threshold and  # 价格超跌（偏离均线≤-0.6%）
            target_vol_multiple >= self.p.vol_multiple and  # 量能放大
            self.daily_trades < self.p.daily_max_trades and  # 未超每日次数
            self.t_pos == 0):
            
            # 计算做T仓位
            t_size = int(self.broker.getvalue() * self.p.t_position_ratio / self.target_close[0])
            if t_size > 0:  # 确保有足够资金
                self.buy(data=self.target_data, size=t_size)
                self.t_pos = t_size
                self.buy_price = self.target_close[0]
                self.daily_trades += 1
                self.hold_minutes = 0
            return

# ----------------------
# 4. 回测执行
# ----------------------
if __name__ == "__main__":
    # 配置参数
    data_dir = r"D:\workspace\xiaoyao\data\stock_minutely_price"  # 你的分钟数据目录
    target_stock = "600570.XSHG"  # 目标股票
    # 你的关联组股票列表（36只）
    correlated_stocks = [
        "000050.XSHE", "000063.XSHE", "000636.XSHE", "000823.XSHE", "000829.XSHE",
        "000851.XSHE", "000977.XSHE", "002008.XSHE", "002036.XSHE", "002049.XSHE",
        "002054.XSHE", "002055.XSHE", "002063.XSHE", "002065.XSHE", "600037.XSHG",
        "600088.XSHG", "600171.XSHG", "600183.XSHG", "600198.XSHG", "600271.XSHG",
        "600360.XSHG", "600386.XSHG", "600410.XSHG", "600446.XSHG", "600460.XSHG",
        "600536.XSHG", "600570.XSHG", "600584.XSHG", "600588.XSHG", "600667.XSHG",
        "600718.XSHG", "600756.XSHG", "600825.XSHG", "600831.XSHG", "600845.XSHG",
        "600880.XSHG"
    ]

    # 加载关联组数据（修复后逻辑）
    target_df, group_data = load_correlated_group_data(correlated_stocks, data_dir, target_stock)

    # 初始化Backtrader引擎
    cerebro = bt.Cerebro()

    # 添加目标股票数据
    target_bt_data = bt.feeds.PandasData(
        dataname=target_df,
        timeframe=bt.TimeFrame.Minutes,
        compression=1
    )
    cerebro.adddata(target_bt_data, name=target_stock)

    # 添加关联组其他股票数据
    for stock, df in group_data.items():
        bt_data = bt.feeds.PandasData(
            dataname=df,
            timeframe=bt.TimeFrame.Minutes,
            compression=1
        )
        cerebro.adddata(bt_data, name=stock)

    # 配置交易参数
    cerebro.broker.setcash(1000000.0)  # 初始资金100万
    cerebro.broker.addcommissioninfo(StampDutyCommissionScheme())  # 佣金+印花税
    cerebro.broker.set_slippage_fixed(0.0004)  # 滑点0.04%（适配关联组温和波动）

    # 添加策略
    cerebro.addstrategy(CorrelatedGroupTStrategy)

    # 添加绩效分析指标
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe', timeframe=bt.TimeFrame.Days)
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
    cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trade_analyzer')

    # 运行回测
    print(f"\n回测开始，初始资金：{cerebro.broker.getvalue():.2f}元")
    results = cerebro.run()
    strat = results[0]

    # 输出回测结果
    print(f"\n回测结束，最终资金：{cerebro.broker.getvalue():.2f}元")
    returns = strat.analyzers.returns.get_analysis()
    print(f"年化收益率：{returns.get('rnorm100', 0):.2f}%")
    
    # 夏普比率异常处理
    sharpe_analysis = strat.analyzers.sharpe.get_analysis()
    sharpe_ratio = sharpe_analysis.get('sharperatio', None)
    print(f"夏普比率：{sharpe_ratio:.2f}" if sharpe_ratio is not None else "夏普比率：无法计算（交易次数不足）")
    
    print(f"最大回撤：{strat.analyzers.drawdown.get_analysis()['max']['drawdown']:.2f}%")

    # 交易细节分析
    trade_analysis = strat.analyzers.trade_analyzer.get_analysis()
    if 'total' in trade_analysis and trade_analysis['total']['closed'] > 0:
        total = trade_analysis['total']['closed']
        won = trade_analysis['won']['total']
        print(f"总做T次数：{total}")
        print(f"做T胜率：{won / total * 100:.2f}%")
        print(f"平均单次收益：{trade_analysis['won']['pnl']['average']:.2f}元")
        print(f"平均单次亏损：{trade_analysis['lost']['pnl']['average']:.2f}元")
    else:
        print("无有效交易记录")

TypeError: Index.intersection() takes from 2 to 3 positional arguments but 35 were given